# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234452283272                   -0.50    7.0         
  2   -7.250076277547       -1.81       -1.39    1.0   5.02ms
  3   -7.250771234929       -3.16       -1.83    1.0   5.18ms
  4   -7.250931197727       -3.80       -1.85    2.0   6.08ms
  5   -7.251335099123       -3.39       -2.79    1.0   5.21ms
  6   -7.251337975014       -5.54       -3.06    2.0   6.53ms
  7   -7.251338715482       -6.13       -3.62    1.0   5.34ms
  8   -7.251338779141       -7.20       -3.92    2.0   6.43ms
  9   -7.251338797122       -7.75       -4.47    1.0   5.53ms
 10   -7.251338798605       -8.83       -5.05    2.0   6.64ms
 11   -7.251338798688      -10.08       -5.48    2.0   6.59ms
 12   -7.251338798702      -10.85       -5.91    1.0   5.84ms
 13   -7.251338798704      -11.69       -6.25    2.0   7.03ms
 14   -7.251338798705      -12.53       -6.74    1.0   5.86ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.049758938946369194
[ Info: Arnoldi iteration step 2: normres = 0.6433844177384684
[ Info: Arnoldi iteration step 3: normres = 0.7547206915446946
[ Info: Arnoldi iteration step 4: normres = 0.31143545636228775
[ Info: Arnoldi iteration step 5: normres = 0.41594925053305665
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.10e-02, 7.13e-02, 3.17e-01, 2.52e-01, 6.05e-02)
[ Info: Arnoldi iteration step 6: normres = 0.23050025176005626
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.58e-03, 1.61e-01, 1.43e-01, 6.77e-02, 1.23e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08472006551297927
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.80e-04, 1.15e-02, 8.92e-03, 2.22e-02, 7.83e-02)
[ Info: Arnoldi iteration step 8: normres = 0.07154634809741824
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (5.4